In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
class Instance:
    O: int  # number of requests
    I: int  # number of items
    A: int  # number of corridors

    u_oi: np.ndarray  # quantity of each item for each request
    u_ai = np.ndarray  # quantity of each item in each corridor

    LB: int  # lower bound on the wave size
    UB: int  # upper bound on the wave size

    def __init__(self, input_file):
        with open(input_file, 'r') as file:
            self.O, self.I, self.A = map(int, file.readline().split())

            # Read orders matrix:
            self.u_oi = np.zeros((self.O, self.I), dtype=int)
            for i in range(self.O):
                data = np.fromstring(file.readline(), dtype=int, sep=' ')
                indices, qtys = data[1::2], data[2::2]
                self.u_oi[i, indices] = qtys

            # Read corridors matrix:
            self.u_ai = np.zeros((self.A, self.I), dtype=int)
            for i in range(self.A):
                data = np.fromstring(file.readline(), dtype=int, sep=' ')
                indices, qtys = data[1::2], data[2::2]
                self.u_ai[i, indices] = qtys

            # Read lower and upper bounds for wave size:
            self.LB, self.UB = map(int, file.readline().split())

# Constants and variables:

In [3]:
input_file = "datasets/a/instance_0020.txt"
inst = Instance(input_file)

# Create a new model
m = gp.Model("meli")

# Create variables
O_ = m.addMVar(inst.O, vtype=GRB.BINARY, name='o')  # wave selected requests
A_ = m.addMVar(inst.A, vtype=GRB.BINARY, name='a')  # wave visited corridors
obj = m.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=inst.u_oi.sum(), name='Obj')  # the objective cost function

m.update()

Set parameter Username
Set parameter LicenseID to value 2615956
Academic license - for non-commercial use only - expires 2026-01-28


# Restrictions:

In [16]:
wave_size = (inst.u_oi.T @ O_).sum()  # total number of items in the wave

# Operational limits on the total number of items for the orders included in the wave:
m.addConstr(wave_size >= inst.LB)
m.addConstr(wave_size <= inst.UB)

# The selected corridors have sufficient storage for each of the items within the wave:
m.addConstrs(O_ @ inst.u_oi[:, i] <= A_ @ inst.u_ai[:, i] for i in range(inst.I))

m.addConstr(obj <= wave_size)  # basic cut
m.update()

# Objective function:

In [17]:
wave_corridors = A_.sum()  # number of used corridors
m.addConstr(obj == wave_size / wave_corridors)
m.setObjective(obj, GRB.MAXIMIZE)

# Model solving:

In [18]:
try:
    m.optimize()

    # Print the found solution:
    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")
    print(f"\nWave size: {wave_size.getValue():g}")
    print(f"Number of corridors: {wave_corridors.getValue():g}")
except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")
except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Freedesktop SDK 23.08 (Flatpak runtime)")

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 11 columns and 46 nonzeros
Model fingerprint: 0x605e8145
Model has 1 general nonlinear constraint (1 nonlinear terms)
Variable types: 1 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+00, 1e+01]
Presolve model has 1 nlconstr
Added 2 variables to disaggregate expressions.
Presolve time: 0.00s
Presolved: 14 rows, 14 columns, 66 nonzeros
Presolved model has 1 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 2 continuous, 12 integer (10 binary)

Root relaxation: objective 9.608696e+00, 18 iterations, 0.00 seconds (0.00 work units)

    No